## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [5]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [6]:
df_lookup = spark.read.json("/user/sprenner/lookup_table_1_official.json")
df_lookup.createOrReplaceTempView("lookup")

In [7]:
spark.sql("SELECT * FROM lookup WHERE distinct_datasets > 1").toPandas()

ADLER32       BYTES                                 CHILD_NAME  \
0     72cc3c8c  10219012    AOD.13709303._001953.pool.root.1            
1     72cc3c8c  10219012    AOD.13709303._001953.pool.root.1            
2     5381a435  18350504    EVNT.13664312._000690.pool.root.1           
3     5381a435  18350504    EVNT.13664312._000690.pool.root.1           
4     29568a7d  55701264    EVNT.13664314._000482.pool.root.1           
5     29568a7d  55701264    EVNT.13664314._000482.pool.root.1           
6     a2dd15eb  55400833    EVNT.13664314._000536.pool.root.1           
7     a2dd15eb  55400833    EVNT.13664314._000536.pool.root.1           
8     043dbd1a  54992338    EVNT.13664314._000635.pool.root.1           
9     043dbd1a  54992338    EVNT.13664314._000635.pool.root.1           
10    c24ae53c  161030287   EVNT.13785124._000224.pool.root.1           
11    c24ae53c  161030287   EVNT.13785124._000224.pool.root.1           
12    81292ae5  14937520    EVNT.13785174._000827.pool.root.1           
13    81292ae5  14937520    EVNT.13785174._000827.pool.root.1           
14    227baf9a  653260273   AOD.13642619._000311.pool.root.1            
15    227baf9a  653260273   AOD.13642619._000311.pool.root.1            
16    f451f146  864991322   AOD.13642632._000844.pool.root.1            
17    f451f146  864991322   AOD.13642632._000844.pool.root.1            
18    018d7cdb  996959837   DAOD_HIGG5D1.13652466._000004.pool.root.1   
19    018d7cdb  996959837   DAOD_HIGG5D1.13652466._000004.pool.root.1   
20    018d7cdb  996959837   DAOD_HIGG5D1.13652466._000004.pool.root.1   
21    018d7cdb  996959837   DAOD_HIGG5D1.13652466._000004.pool.root.1   
22    d088890f  42658396    DAOD_SUSY5.13462844._000097.pool.root.1     
23    d088890f  42658396    DAOD_SUSY5.13462844._000097.pool.root.1     
24    d0bfa435  40827200    DAOD_SUSY5.13462844._000207.pool.root.1     
25    d0bfa435  40827200    DAOD_SUSY5.13462844._000207.pool.root.1     
26    33b5049e  916903877   AOD.13776843._000078.pool.root.1            
27    33b5049e  916903877   AOD.13776843._000078.pool.root.1            
28    33b5049e  916903877   AOD.13776843._000078.pool.root.1            
29    33b5049e  916903877   AOD.13776843._000078.pool.root.1            
...        ...        ...                                ...            
6298  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6299  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6300  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6301  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6302  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6303  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6304  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6305  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6306  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6307  2129cefb  868820011   AOD.13642632._001845.pool.root.1            
6308  17a5690e  42596610    DAOD_SUSY5.13462844._000120.pool.root.1     
6309  17a5690e  42596610    DAOD_SUSY5.13462844._000120.pool.root.1     
6310  b2679c32  970182653   AOD.13634605._000011.pool.root.1            
6311  b2679c32  970182653   AOD.13634605._000011.pool.root.1            
6312  323db8da  1064846290  AOD.13641049._000031.pool.root.1            
6313  323db8da  1064846290  AOD.13641049._000031.pool.root.1            
6314  a6ac1d70  1058055601  AOD.13641049._000082.pool.root.1            
6315  a6ac1d70  1058055601  AOD.13641049._000082.pool.root.1            
6316  e50d92f8  1042463779  AOD.13641049._000089.pool.root.1            
6317  e50d92f8  1042463779  AOD.13641049._000089.pool.root.1            
6318  8ec29ce4  1067497013  AOD.13641049._000103.pool.root.1            
6319  8ec29ce4  1067497013  AOD.13641049._000103.pool.root.1            
6320  73868cee  883252099   AOD.13776843._000013.pool.root.1     

In [8]:
df_april = spark.read.json("/user/sprenner/training_official_april.json")
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["scope"], df_april["filename"]))
df_april = df_april.withColumn("fileScope", df_april["scope"])
df_april = df_april.drop("scope")
df_april.createOrReplaceTempView("april")

In [14]:
df_lookup = df_lookup.dropDuplicates(['BYTES','CHILD_NAME','CHILD_SCOPE','CHILD_TYPE','DID_TYPE','NAME','SCOPE'])
df_lookup.createOrReplaceTempView("lookup")
df_lookup = spark.sql("SELECT * FROM lookup WHERE SCOPE != 'hc_test' AND distinct_datasets == 1") # zum checken distinct_datasets rausnehmen
df_lookup.createOrReplaceTempView("lookup")
multiple = spark.sql("SELECT join_key_april AS join_key_april_multi, COUNT(*) as count\
           FROM lookup \
           GROUP BY join_key_april\
           HAVING count > 1 \
           ORDER BY 2 DESC")
multiple.toPandas()

Empty DataFrame
Columns: [join_key_april_multi, count]
Index: []

In [10]:
df_lookup.describe().toPandas()

summary   ADLER32                 BYTES                          CHILD_NAME  \
0  count   527263    527263                527263                               
1  mean    Infinity  4.0630838745106333E8  None                                 
2  stddev  NaN       9.272619894227926E8   None                                 
3  min     00000425  10000                 AOD.05371635._000001.pool.root.1     
4  max     fff0cad9  999995790             log.13787275._000072.job.log.tgz.1   

  CHILD_SCOPE CHILD_TYPE             CREATED_AT DID_TYPE              EVENTS  \
0  527263      527263     527263                 527263   446585               
1  None        None       1.5087930319983213E12  None     29534.996435169116   
2  None        None       1.7975082663408134E10  None     585485.8208299025    
3  data10_hi   F          1301113739000          D        0                    
4  mc16_valid  F          1524971076000          D        9999500              

  LENGTH  \
0  1       
1  0.0     
2  NaN     
3  0       
4  0       

                                                                                                                 NAME  \
0  527263                                                                                                               
1  None                                                                                                                 
2  None                                                                                                                 
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456_tid289065_00                                                 
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210_tid13765955_00   

  RULE_EVALUATION       SCOPE            UPDATED_AT  account  \
0  1               527263      527263                527263    
1  0.0             None        1.50879308628084E12   None      
2  NaN             None        1.797507988241306E10  None      
3  0               data10_hi   1301113739000         aagaard   
4  0               mc16_valid  1524971079000         zzheng    

            accountHash         clientState       clientStateHash  \
0  527263                527263              527263                 
1  2.78300204514049E8    None                -9.266224034762633E8   
2  1.0874914213535457E9  None                1.745316993917929E8    
3  -2127257360           CannotAuthenticate  -1922357674            
4  2143592175            SourceNotFound      1269532611             

                                                                                               dataset  \
0  527263                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

           datasetHash datasetScope      datasetScopeHash dataset_version  \
0  527263               527263       527263                527263           
1  6.016572373083262E7  None         -6.539749138209945E8  None             
2  1.182422604119735E9  None         8.519420071680802E8   None             
3  -2147401420          data10_hi    -2057006107           c1130_m1847      
4  2147445885           mc16_valid   2109319881            x557_m1947       

    dataset_versionHash datatype          datatypeHash distinct_datasets  \
0  527263                527263   527263                527263             
1  3.79031105190772E8    None     4.935396855312131E8   1.0                
2  1.2234715758065019E9  None     1.1402421581859827E9  0.0                
3  -2146716518           AOD  

In [11]:
#df_lookup = df_lookup.withColumn('dataset_lookup',when(col("distinct_datasets").isNull(), df_lookup["dataset"]).otherwise(df_lookup["NAME"]))

In [12]:
df_datasets_lookup = df_lookup.select("NAME","SCOPE","DID_TYPE","CHILD_NAME","CHILD_SCOPE", "join_key_lookup")

In [13]:
df_datasets_lookup.write.format("json").save("/user/sprenner/lookup_table_2_official.json")